In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import joblib

# --- 1. Load Assets ---
# Using the Stacking Model (digital_twin_assets.pkl) for high-resolution contour stability
assets = joblib.load('digital_twin_assets.pkl')
model = assets['model']
materials_db = assets['materials']

# --- 2. Define Slices & Grid ---
# Radius values from Sobol bounds [0.8, 1.5]
radii_slices = [0.8, 1.15, 1.5]
slice_labels = ['Focused (0.8mm)', 'Nominal (1.15mm)', 'Defocused (1.5mm)']

# Grid generation
power_range = np.linspace(50, 300, 50)
speed_range = np.linspace(5, 50, 50)
P_grid, V_grid = np.meshgrid(power_range, speed_range)

# Sub-figure labels mapping
subfigure_labels = ['(a)', '(b)', '(c)', '(d)', '(e)']

# --- 3. Plotting Loop ---
# We enumerate to get the index (0, 1, 2...) for the (a), (b), (c) labels
for idx, (mat_name, props) in enumerate(materials_db.items()):
    if idx >= len(subfigure_labels): break # Safety break

    label = subfigure_labels[idx] # e.g., "(a)"
    rho, k, cp = props

    # Initialize Figure (3 columns for the 3 radius slices)
    fig, axes = plt.subplots(1, 3, figsize=(18, 5), dpi=300, sharey=True)

    # Plot each slice
    for i, radius in enumerate(radii_slices):
        ax = axes[i]

        # Dynamic Input Grid
        grid_df = pd.DataFrame({
            'Power_W/mm2': P_grid.ravel(),
            'Speed_mm/s': V_grid.ravel(),
            'Radius_mm': radius,
            'Density': rho,
            'Conductivity': k,
            'Cp': cp
        })

        # Predict
        preds = model.predict(grid_df)
        temp_preds = preds[:, 3] # Index 3 = Max Temp
        Z = temp_preds.reshape(P_grid.shape)

        # Contour Plot
        contour = ax.contourf(P_grid, V_grid, Z, levels=25, cmap='magma')

        # Safety Threshold (200C)
        if Z.min() < 200 < Z.max():
            cs = ax.contour(P_grid, V_grid, Z, levels=[200], colors='cyan', linewidths=2, linestyles='--')
            ax.clabel(cs, fmt='200°C', fontsize=10)

        # Slice Title
        ax.set_title(f"{slice_labels[i]}", fontsize=12)
        ax.set_xlabel("Power (W/mm²)")
        if i == 0: ax.set_ylabel("Speed (mm/s)")

    # Shared Colorbar
    cbar = fig.colorbar(contour, ax=axes.ravel().tolist(), pad=0.02)
    cbar.set_label(f'Max Temp (°C)', rotation=270, labelpad=15)

    # Main Figure Title (The one you requested)
    full_title = f"Figure 17 {label}: Effect of Beam Radius on Process Window ({mat_name})"
    plt.suptitle(full_title, fontsize=16, fontweight='bold', y=1.05)

    # Save File
    # Filename format: Figure_16_a_CoCrFeNiMn.pdf
    clean_label = label.replace('(', '').replace(')', '') # remove parens for filename
    filename = f"Figure_17_{clean_label}_{mat_name}.pdf"

    plt.savefig(filename, bbox_inches='tight')
    plt.close()

    print(f"Generated: {filename} -> {full_title}")

print("\nSUCCESS: All 5 sub-figures generated with requested naming convention.")

Generated: Figure_17_a_CoCrFeNiMn.pdf -> Figure 17 (a): Effect of Beam Radius on Process Window (CoCrFeNiMn)
Generated: Figure_17_b_AlCoCrFeNi.pdf -> Figure 17 (b): Effect of Beam Radius on Process Window (AlCoCrFeNi)
Generated: Figure_17_c_FeCrNiTiAl.pdf -> Figure 17 (c): Effect of Beam Radius on Process Window (FeCrNiTiAl)
Generated: Figure_17_d_NbSiTaTiZr.pdf -> Figure 17 (d): Effect of Beam Radius on Process Window (NbSiTaTiZr)
Generated: Figure_17_e_HfNbTiZr.pdf -> Figure 17 (e): Effect of Beam Radius on Process Window (HfNbTiZr)

SUCCESS: All 5 sub-figures generated with requested naming convention.
